In [1]:
import sys
sys.path.append(r"../")
#sys.path.append(r"../utils/data_manipulation")
import pandas as pd
from utils.data_manipulation.data_imputation import impute_from_column

In [2]:
redcap_event_names = [
#    'intake_arm_1',
    'control_6month_arm_1',
    'pre_treatment_arm_1',
    '5th_session_arm_1',
    'post_treatment_arm_1',
    'followup_3month_arm_1',
    'control_3month_arm_1',
    'control_5weeks_arm_1',
#    'corona_arm_1',
#    'er_arm_1',
    'followup_12month_arm_1',
    'er_one_week_arm_1',
    'control_12month_arm_1', 
    'control_9month_arm_1'
]

sci_af_ca = [
    'sci_af_ca_1',
    'sci_af_ca_2',
    'sci_af_ca_3',
    'sci_af_ca_4',
    'sci_af_ca_5',
    'sci_af_ca_6',
    'sci_af_ca_7',
    'sci_af_ca_8',
    'sci_af_ca_9',
    'sci_af_ca_10',
    'sci_af_ca_11',
    'sci_af_ca_12',
    'sci_af_ca_13',
    'sci_af_ca_14',
    'sci_af_ca_15',
    'sci_af_ca_16',
    'sci_af_ca_17',
    'sci_af_ca_18',
    'sci_af_ca_19',
    'sci_af_ca_20',
    'sci_af_ca_21',
    'sci_af_ca_22',
    'sci_af_ca_23',
    'sci_af_ca_24',
    'sci_af_ca_25',
    'sci_af_ca_26',
    'sci_af_ca_27',
    'sci_af_ca_28',
    'sci_af_ca_29',
    'sci_af_ca_30',
    'sci_af_ca_31',
    'sci_af_ca_32',
    'sci_af_ca_33',
    'sci_af_ca_34',
    'sci_af_ca_35',
    'sci_af_ca_36',
    'sci_af_ca_37',
    'sci_af_ca_38',
    'sci_af_ca_39',
    'sci_af_ca_40'
]


In [3]:
df = pd.read_csv(r"..\creating_the_clinic_dataset\preprocessed_data\Depp_Clinic_data_reshaped.csv")

columns = ["id", "age_child_pre"]

C:\Users\nogur\AppData\Local\Temp\ipykernel_8088\2596220389.py:1: DtypeWarning: Columns (0,2,11,12,27,36,43,49,69,72,83,92,108,137,154,180,221,263,287,300,308,348,370,381,418,436,441,442,456,472,474,475,478,483,485,490,492,501,502,521,537,539,548,567,576,610,632,669,680,690,727,732,733,739,744,749,754,755,757,758,761,766,768,770,773,775,784,785,804,820,822,831,850,859,893,915,952,963,973,1010,1015,1016,1022,1027,1032,1037,1038,1040,1041,1044,1058,1067,1082,1103,1105,1125,1141,1146,1162,1164,1213,1215,1217,1270,1272,1273,1274,1280,1281,1282,1283,1336,1393,1396,1397,1412,1414,1416,1419,1435,1437,1438,1439,1445,1446,1447,1448,1501,1558,1561,1562,1568,1572,1584,1591,1592,1623,1658,1677,1687,1704,1730,1771,1813,1837,1850,1920,1931,1999,2006,2022,2024,2025,2071,2126,2160,2182,2219,2230,2283,2289,2305,2307,2308,2354,2409,2443,2465,2502,2513,2566,2588,2590,2591,2594,2634,2675,2691,2696,2714,2775,2779,2886,2943,2946,2947,2962,2964,2966,2969,3051,3108,3111,3112,3113,3118,3122,3124,3125,3126,3131

## Intake time

In [4]:
df["intake_date"] =  pd.to_datetime(df["sciafca_timestamp_intake_arm_1"], errors='coerce')
columns.append("intake_date")

## ER time

In [5]:
df = impute_from_column(df, "complaint_date", "chameleon_suicide_er_date_stu_intake_arm_1")
df["ER_date_time1"] = df["complaint_date"]
columns.append("ER_date_time1")

###  ER date

In [6]:
time2_er_dates = [f"chameleon_suicide_er_date_stu_{event}" for event in redcap_event_names]
for er_time in time2_er_dates:
    df[er_time] = pd.to_datetime(df[er_time], errors='coerce')
df["ER_date_target"] = df[time2_er_dates].min(axis=1)
columns.append("ER_date_target")

## Psych ward

In [7]:
time2_psych_dates = [f"chameleon_psychiatric_date_stu_{event}" for event in redcap_event_names]
for psych_time in time2_psych_dates:
    df[psych_time] = pd.to_datetime(df[psych_time], errors='coerce')
    
df["psychiatric_date_target"] = df[time2_psych_dates].min(axis=1)
columns.append("psychiatric_date_target")

## Attempt


In [8]:
time2_visit_date_dates = [f"visit_date_stu_{event}" for event in redcap_event_names]
time2_is_attempt = [f"chameleon_attempt_stu_{event}" for event in redcap_event_names]

In [9]:
for visit_date, is_attempt in zip(time2_visit_date_dates, time2_is_attempt):
    df.loc[df[is_attempt] != 1, visit_date] = pd.NaT
    df[visit_date] = pd.to_datetime(df[visit_date], errors='coerce')

In [10]:
df["suicide_attempt_date_target"] = df[time2_visit_date_dates].min(axis=1)
columns.append("suicide_attempt_date_target")

## Save

In [11]:
sci_intake = [f"{q}_intake_arm_1" for q in sci_af_ca]
columns.extend(sci_intake)

In [12]:
df[columns].to_csv("data_for_survival_analysis.csv")